# Create a nanowire

Create a nanowire from original material by creating a supercell and cutting a cylinder.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## 1. Prepare the Environment
### 1.1. Set up transformation parameters 

In [ ]:
XY_SUPERCELL_MATRIX = [
    [3, 0], 
    [0, 3],
] 
HEIGHT = 3

MILLER_INDEX = (0,0,1)  # Miller index of the nanowire direction
# Cylinder parameters in crystal units of supercell
RADIUS = 0.25
ALIGN_ALONG_X = True

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("", "../../config.yml")

### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0]

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize(material, repetitions=[3,3,3], rotation="0x")
visualize(material, repetitions=[3,3,3], rotation="-90x")

## 2. Create nanowire
### 2.1. Create a slab and cut a cylinder

In [ ]:
from mat3ra.made.tools.build.supercell import create_supercell
from mat3ra.made.tools.modify import filter_by_cylinder
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration

slab_config = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDEX,
    thickness=HEIGHT,
    vacuum=0,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX,
)
supercell = create_slab(slab_config)
nanowire_z = filter_by_cylinder(supercell, radius=RADIUS)
rotate_90_degree_matrix = [[0, 0, 1], [0, 1, 0], [-1, 0, 0]]
nanowire_x = create_supercell(nanowire_z, matrix=rotate_90_degree_matrix)
nanowire = nanowire_x if ALIGN_ALONG_X else nanowire_z

## 3. Visualize the result

In [ ]:
visualize(nanowire, repetitions=[1, 1, 1], rotation="0x")
visualize(nanowire, repetitions=[1, 1, 1], rotation="-90x")

# 4. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(nanowire)